# Cubic Splines

flexibiliyとsimplicityが大きいが、連続な1次導関数、2次導関数を持つ。  
そのため、関数も1次、2次導関数も十分に近似できる。  

## The Basis Function

The basis functions for an n-degree cubic spline with evenly spaced breakpoints on the interval [a, b]

The n-2 breakpoints　　

$$𝑣_𝑗=𝑎+ℎ(𝑗−1),  𝑗=1, 2,…, 𝑛−2$$

$$ℎ=\frac{𝑏−𝑎}{𝑛−3}$$

In [2]:
function breakpoints(a::Real, b::Real, n::Real)
    v = zeros(n-2)
    h = (b - a) / (n - 3)
    for j in 1:n-2
        v[j] = a + h * (j - 1)
    end
    return v
end

breakpoints (generic function with 1 method)

In [16]:
x = collect(a:(b - a)/n:b)


LoadError: LoadError: UndefVarError: a not defined
while loading In[16], in expression starting on line 1

また、曲率が最も大きい範囲にbreakpointsを集めれば曲率による近似の困難さをうまく乗り切れることがしばしばある。

Runge’s function 

$$𝑓(𝑥)= (1 +25𝑥^2)^{-1}, 𝑥∈[−5, 5]$$

Runge's functionが高い曲率を示している範囲（0付近）にbreakpointsを集めた場合、breakpointsを均等に配置した場合よりもApproximation errorsは小さくなる。

通常k次のスプラインはbreakpointsにおいてk-1次まで連続な導関数を持つ。  
q個のbreakpointsを積み重ねることで、そのbreakpointにおいてk-q次まで連続な導関数を持つことができる。  
ある点でkinkを持つことが分かっていればbreakpointsの積み重ねが役立つ。

### important and useful properties

basis functionの値が0にならない点は限られているので、sparse matrix methodを使ってspline interpolation matricesを記憶・操作できる。  
sparse matrixとは成分のほとんどが0である行列のこと。

スプラインの値は有界なのでscaling effectsによって数値計算上の困難が起こる可能性は低い

### Inverse

特に行列が大きいとき、spline interpolation matrixを逆行列にするとsparsity構造が壊れてしまう。

# 6.4   Piecewise Linear Basis Functions  

近似される関数がスムーズでなく不連続のときでさえlinear splineを使えばかなり近似できる。  
ただし、導関数は区分分けされた定数関数で、不連続。

導関数近似の有力候補はfinite-difference approximationsを使ったlinear spline

もともとのn個のbreakpointsの中点に新しくn-1個のbreakpointをつくる。

$$𝑧_𝑖=  \frac{𝑣_𝑖 + 𝑣_{𝑖+1}}{2},  𝑖=1, …,𝑛-1 $$

新しいbreakpointsにおける微分の近似

$$𝑓^′ (𝑧_𝑖 ) \approx  \frac{𝑓(𝑣_{𝑖+1}) − 𝑓(𝑣_𝑖)}{𝑣_{𝑖+1} − 𝑣_𝑖}$$

$z_i$の間または外側の値は線形補間または外挿によって得る

積分近似ではもともとのbreakpointsの範囲を超えて外挿した値も使う

$$𝑧_1= \frac{3𝑣_1−𝑣_2}{2}$$  

$$𝑧_{𝑛+1}= \frac{3𝑣_𝑛 − 𝑣_{𝑛−1}}{2}$$

積分

$$F(x) = \int_{v_1}^{x}f(x)dx$$

を新しいbreakpointsで近似

$$F(z_i) = F(z_{i-1}) + (z_i - z_{i-1})f(v_{i-1})$$

ただし

$$F(z_1) = \frac{1}{2}(v_1 - v_2)f(v_1)$$

この定義によって得られる積分近似はもともとのbreakpointsにおいては台形則によって得られる近似に等しい。

$$\int_{𝑣_𝑖}^{𝑣_{𝑖+1}}f(x)dx \approx \frac{1}{2}(𝑣_{𝑖+1}−𝑣_𝑖)[𝑓(𝑣_{𝑖+1})+𝑓(𝑣_𝑖)]$$

In [ ]:
検証？

これらの近似関数によって得られる"derivatives"や"integrals"は近似とみなしてよいが、polynomialやspline関数の場合はダメ。  
しかし、もしも$\phi(X)c$の導関数を$\phi^{'}(X)c$のように定義したならばこれらの近似を使って得られるpiecewise linear functionsも同様に微分作用・積分作用について閉じている。（$\phi^{'}(X)c$集合に属する？）  

splineでは微分・積分はpiecewise segmentsの次数を増減させ、breakpoint sequenceは変化させない。  
piecewise linear familyは逆。  
finite difference operatorsを使った計算が楽。  